In [ ]:
import psycopg2

In [ ]:
def create_database():
    """
    This function creates a new database which new data will be inserted into. 
    1. create a connection to the local existing database in postgres. 
    2. create a cursor and create a new database using that cursor. 
    3. close the connection and the cursor and create a new connection and a new cursor to the new database that was just created.

    return: the final connection and the final cursor with which you can execute queries
    """
    
    try:
        conn = psycopg2.connect('host = localhost dbname = postgres user=postgres password = 1011')
    except psycopg2.Error as e:
        print("could not connect to postgres database")
        print(e)
    
    try: 
        cur = conn.cursor()
    except psycopg2.Error as e:
        print("could not create cursor")
        print(e)
        
    conn.set_session(autocommit=True)
    
    
#     Create a new database and connect to that database

    try:
        cur.execute("CREATE DATABASE project1db")
    except psycopg2.Error as e:
        print("could not create a database")
        print(e)
    
    conn.close()
    cur.close()
    
    try:
        conn = psycopg2.connect('host = localhost dbname = project1db user=postgres password = 1011')
    except psycopg2.Error as e:
        print("could not connect to postgres database")
        print(e)
        
    try: 
        cur = conn.cursor()
    except psycopg2.Error as e:
        print("could not create cursor")
        print(e)
        
    conn.set_session(autocommit=True)
    
    return conn, cur

In [ ]:
conn, cur = create_database()

In [ ]:
def create_table(cur, tablename, columns_with_datatypes):
    """
    This function creates a table in the database you created with create_database function.
    
    Paramaters:
    cur: the cursor you created which is custed to execute any query
    tablename: the name of a new table you will create as a string
    (columns_with_datatypes): a string of all the columns with data types respectively
    
    Example:
    >>> create_table(cur, 'example_table', ("column1 int, column2 varchar"))
    >>> nothing returns, but the table named 'example_table' is created into your database.
    """
                 
    try:
        cur.execute(f"CREATE TABLE IF NOT EXISTS {tablename} ({columns_with_datatypes})")
    except psycopg2.Error as e:
        print(f"could not create a table: {tablename}")
        print(e)

In [ ]:
create_table(cur, 'expenditures', ('expenditure_id varchar, household_id varchar, year int, month int, product_code varchar, cost numeric, gift int, is_training int'))

In [ ]:
create_table(cur, 'household_members', ('household_id varchar, year int, marital varchar, sex varchar, age int, work_status varchar'))

In [ ]:
create_table(cur, 'households', ('household_id varchar, year int, income_rank numeric, income_rank_1 numeric, income_rank_2 numeric, income_rank_3 numeric, income_rank_4 numeric, income_rank_5 numeric, income_rank_mean numeric, age_ref int '))

## Check the csv files with data that you want to insert into your new tables 

In [ ]:
import pandas as pd

In [ ]:
expenditures_df = pd.read_csv('expenditures.csv')
expenditures_df.head()

In [ ]:
household_members_df = pd.read_csv('household_members.csv')
household_members_df.head()

In [ ]:
households_df = pd.read_csv('households.csv')
households_df.head()

In [ ]:
def lower_column_names(df):
    """
    This function lowers the columns names.
    
    Paramater:
    df: the dataframe you want to lower columns from.
    
    Example:
    example_df has columns COLUMN_1, COLUMN_2
    >>> lower_column_names(example_df)
    >>> index(['column_1', 'column_2'], dtype='object')
    """
    
    lowered_columns = [column.lower() for column in df.columns.tolist()]
    df.columns = lowered_columns
    return df.columns

In [ ]:
lower_column_names(expenditures_df)

In [ ]:
lower_column_names(household_members_df)

In [ ]:
lower_column_names(households_df)

## add keys, default values, null values ..etc

### Default Values: 
#### - expenditures: 
####      - is_training: default value= 1
#### - household_members: 
####     - work_status: default null

### Not null:
#### all columns except for the ones with default values should not have null values.

### Keys:
#### - expenditures: primary key=expenditure_id, foreign key= household_id
#### - household_members: foreign key= household_id, key(index, btree)= household_id
####  - households: primary key= household_id, key(index, btree)=household_id
     


In [ ]:
def setDefaultValues(tablename, columnname, defaultValue):
     """
    Sets a default value for a specified column in a PostgreSQL table.

    Parameters:
    tablename : str
        The name of the table in which the column exists.
        
    columnname : str
        The name of the column to set the default value for.
        
    defaultValue : str, int, float, etc.
        The default value to be assigned to the column when no explicit value is provided.
        The type of `defaultValue` should be compatible with the column's data type.

    Returns:
    None
        This function does not return a value. If an error occurs during the operation, 
        it will be caught and printed.

    Notes:
    - The `DEFAULT` constraint ensures that if no value is provided during an insert operation, 
      the specified default value is automatically used.
    - Ensure that the default value is appropriate for the column's data type (e.g., a string 
      for a text column, or a number for an integer column).
    - This function only sets the default for future insertions. Existing rows will not be updated 
      with the default value.
    - Any `psycopg2.Error` encountered will be printed for debugging.

    Example:
    >>> setDefaultValues('households', 'region', "'North America'")
    """
    try:
        cur.execute(f'ALTER TABLE {tablename} ALTER COLUMN {columnname} SET DEFAULT {defaultValue}')
    except psycopg2.Error as e:
        print(e)

In [ ]:
setDefaultValues('expenditures', 'is_training', '1')

In [ ]:
setDefaultValues('household_members', 'work_status', 'NULL')

In [ ]:
def setNotNull(tablename, columnname):
    """
    Sets a NOT NULL constraint on a specified column in a PostgreSQL table, ensuring that
    the column cannot contain null values.

    Parameters:
    tablename : str
        The name of the table in which the column exists.
        
    columnname : str
        The name of the column to enforce the NOT NULL constraint on.

    Returns:
    None
        This function does not return a value. If an error occurs, it will be caught and
        printed for debugging.

    Notes:
    - The NOT NULL constraint enforces that all entries in the specified column have a value.
    - Applying NOT NULL to a column with existing null values will cause an error. Ensure that
      the column has no nulls before running this command or handle the null values as needed.
    - Any `psycopg2.Error` encountered will be printed.

    Example:
    >>> setNotNull('households', 'household_id')
    """
    try:
        cur.execute(f"ALTER TABLE {tablename} ALTER COLUMN {columnname} SET NOT NULL")
    except psycopg2.Error as e:
        print(e)

In [ ]:
# setNotNull('expenditures', 'expenditure_id')
setNotNull('expenditures', 'household_id')
setNotNull('expenditures', 'year')
setNotNull('expenditures', 'month')
setNotNull('expenditures', 'product_code')
setNotNull('expenditures', 'cost')
setNotNull('expenditures', 'gift')

In [ ]:
setNotNull('household_members', 'household_id')
setNotNull('household_members', 'year')
setNotNull('household_members', 'marital')
setNotNull('household_members', 'sex')
setNotNull('household_members', 'age')

In [ ]:
# setNotNull('households', 'household_id')
setNotNull('households', 'year')
setNotNull('households', 'income_rank')
setNotNull('households', 'income_rank_1')
setNotNull('households', 'income_rank_2')
setNotNull('households', 'income_rank_3')
setNotNull('households', 'income_rank_4')
setNotNull('households', 'income_rank_5')
setNotNull('households', 'income_rank_mean')
setNotNull('households', 'age_ref')

In [ ]:
# By default, PostgreSQL uses the format table-name_pkey as the default name for the primary key constraint.
def setPrimaryKeys(tablename, columnname):
    """
    Sets a primary key constraint on a specified column in a PostgreSQL table.

    Parameters:
    tablename : str
        The name of the table where the primary key constraint will be added.
        
    columnname : str
        The name of the column to be designated as the primary key.

    Returns:
    None
        This function does not return a value. It executes an SQL command to add a primary key 
        constraint on the specified column.

    Notes:
    - By default, PostgreSQL names the primary key constraint in the format `{tablename}_pkey`.
    - Primary keys enforce uniqueness and ensure that no null values are allowed in the specified column.
    - Ensure that the column specified is suitable as a primary key (unique, non-nullable).
    - An error may occur if a primary key already exists on the table or if duplicate values are 
      present in the column. This should be handled based on the specific requirements of the 
      table design.

    Example:
    >>> setPrimaryKeys('households', 'household_id')
    """
    try:
        cur.execute(f"ALTER TABLE {tablename} ADD CONSTRAINT PRIMARY KEY ({columnname})")
    except psycopg2.Error as e:
        print(e)

In [ ]:
setPrimaryKeys('expenditures', 'expenditure_id')

In [ ]:
setPrimaryKeys('households', 'household_id')

In [ ]:
def setForiegnKeys(tablename, columnname, reference_tablename, reference_columnname):
    cur.execute(f"ALTER TABLE {tablename} ADD FOREIGN KEY ({columnname}) REFERENCES {reference_tablename}({reference_columnname})")

In [ ]:
setForiegnKeys('expenditures', 'household_id', 'households', 'household_id')

In [ ]:
setForiegnKeys('household_members', 'household_id', 'households', 'household_id')

In [ ]:
# By default, PostgreSQL creates a B-tree index.
def createIndex(tablename, columnname):
    """
    Creates an index on a specified column in a PostgreSQL table to improve query performance.

    Parameters:
    tablename : str
        The name of the table on which the index will be created.
        
    columnname : str
        The name of the column to index within the specified table.

    Returns:
    None
        This function does not return a value. It executes an SQL command to create an index
        on the specified column.

    Notes:
    - Indexing can improve the performance of queries that frequently search or filter by
      the specified column.
    - The naming convention for the index is `{tablename}_index`, which helps keep index names
      unique and identifiable.
    - Consider the performance trade-offs with indexing, as it may slow down insertions and
      updates on large tables.

    Example:
    >>> createIndex('households', 'household_id')
    """
    try:
        cur.execute(f"CREATE INDEX {tablename}_index ON {tablename}({columnname})")
    except psycopg2.Error as e:
        print(e)

In [ ]:
createIndex('household_members', 'household_id')

In [ ]:
createIndex('households', 'household_id')

## Check configurations by inserting sample values
### we need to insert values in households first since the other two tables reference household table.

In [ ]:
try:
    cur.execute("INSERT INTO households VALUES ('3111041', '2015', '0.304403', '0.144808', '0.142726', '0.143244', '0.14218','0.138187','0.126976','66')")
except psycopg2.Error as e:
    print(e)

## Check default value (is_training should be set to 1 if no value is given)

In [ ]:
try:
    cur.execute("INSERT INTO expenditures VALUES ('1', '3111041', '2015', '1', '10210', '3.89', '0')")
except psycopg2.Error as e:
    print(e)

In [ ]:
try:
    cur.execute("SELECT * FROM expenditures")
except psycopg2.Error as e:
    print(e)

cur.fetchall()

# Check default value ( work_status should be set to NULL if no value is given)

In [ ]:
try:
    cur.execute("INSERT INTO household_members VALUES ('3111041', '2015', '1', '1', '66')")
except psycopg2.Error as e:
    print(e)

In [ ]:
try:
    cur.execute("SELECT * FROM household_members")
except psycopg2.Error as e:
    print(e)

cur.fetchall()

In [ ]:
def deleteAll(tablename):
    """
    Deletes all rows from the specified PostgreSQL table.

    Parameters:
    tablename : str
        The name of the table from which all rows will be deleted.
        
    Returns:
    None
        This function does not return a value. Any errors encountered during deletion
        are caught and printed.

    Notes:
    - Deletion order is crucial if foreign key constraints are in place. To avoid
      foreign key violations, ensure that tables are deleted in a sequence where 
      referencing tables are cleared before tables they reference.
    - This function should be used carefully in production environments, as it will 
      remove all data from the specified table.
    - Any `psycopg2.Error` encountered will be printed for debugging purposes.

    Example:
    >>> deleteAll('household_members')
    """
    
    try: 
        cur.execute(f"DELETE FROM {tablename}")
    except psycopg2.Error as e:
        print(e)

In [ ]:
deleteAll('expenditures')
deleteAll('household_members')
deleteAll('households')

## Insert all data from csv files into tables 

In [ ]:
import psycopg2.extras as extras 

In [ ]:
def insertValues(tablename, df):
        """
    Inserts values from a Pandas DataFrame into a specified PostgreSQL table.

    Parameters:
    tablename : str
        The name of the PostgreSQL table where data will be inserted.
        
    df : pandas.DataFrame
        The DataFrame containing the data to insert into the table. Each row in
        the DataFrame represents a row to insert, and each column corresponds to
        a column in the database table.

    Returns:
    None
        This function does not return a value. If an error occurs during insertion,
        it will be caught and printed.

    Notes:
    - This function converts the DataFrame into a list of tuples, matching the row structure
      of the table.
    - The function uses `psycopg2.extras.execute_values` for efficient bulk insertion.
    - If a `psycopg2.Error` is encountered, the error message will be printed.
      
    Example:
    >>> insertValues('my_table', df)
    """
        
    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ','.join(list(df.columns))
    query = "INSERT INTO %s(%s) VALUES %%s" %(tablename, cols)
    
    try:
        extras.execute_values(cur, query, tuples)
    except psycopg2.Error as e:
        print(e)

In [ ]:
insertValues('households', households_df)

In [ ]:
# insertValues('expenditures', expenditures_df)

In [ ]:
# insertValues('household_members', household_members_df)

## Error: There are household_id from expenditure_df that don't exist in households_df. These need to be removed. (How we handle those values depends on the business needs. Alternatively, we could add that household_id into households table if that cannot be removed).

In [ ]:
household_ids_to_remove_from_expenditures = set(expenditures_df['household_id']).difference(households_df['household_id'])

In [ ]:
household_ids_to_remove_from_household_members = set(household_members_df['household_id']).difference(households_df['household_id'])

In [ ]:
expenditures_df = expenditures_df.loc[~expenditures_df['household_id'].isin(household_ids_to_remove_from_expenditures)]

In [ ]:
household_members_df = household_members_df.loc[~household_members_df['household_id'].isin(household_ids_to_remove_from_household_members)]

In [ ]:
expenditures_df

In [ ]:
(~expenditures_df['household_id'].isin(household_ids_to_remove_from_expenditures)).sum()

In [ ]:
(~household_members_df['household_id'].isin(household_ids_to_remove_from_household_members)).sum()

### You go back to insertValues part and execute that function for expenditures and household_members tables and check if the number of rows inserted are correct. 

### expenditures: 9 rows
### household_members: 292 rows

In [ ]:
try:
    cur.execute("SELECT COUNT(*) FROM expenditures")
except psycopg2.Error as e:
    print(e)

In [ ]:
cur.fetchall()

In [ ]:
try:
    cur.execute("SELECT COUNT(*) FROM household_members")
except psycopg2.Error as e:
    print(e)

In [ ]:
cur.fetchall()

In [ ]:
cur.close()
conn.close()